In [2]:
from collections import defaultdict
import bbhash
from bbhash_table import BBHashTable
import random
import time
from rbloom import Bloom

In [3]:
expected_error_rates = [pow(2, -7), pow(2,-8), pow(2, -10)]
max_set_sizes = [1000, 10000, 100000, 1000000, 10000000]
finger_print_bits = [7, 8, 10]
print(expected_error_rates)
sets_k = []
sets_k_prime = []
bloom_filters = []

[0.0078125, 0.00390625, 0.0009765625]


In [4]:

# create the sets k and k'
for set_size in max_set_sizes:
    all_keys = [random.randint(100, 10000000000) for i in range(set_size)]
    sets_k_prime.append(all_keys)
    sets_k.append(all_keys[:(int(set_size * .5))])
    print("finished set size " + str(set_size))

print("finished creating all sets") 

finished set size 1000
finished set size 10000
finished set size 100000
finished set size 1000000
finished set size 10000000
finished creating all sets


In [4]:
# Task 1
# create the bloom filters and fill them
print("error rate, set size, num false positives, time elapsed, size in bits of bf")
for error_rate in expected_error_rates:
    # print("error rate: " + str(error_rate))
    for i in range(len(max_set_sizes)):
        max_set_size = max_set_sizes[i]
        # print("set size: " + str(set_size))
        # creating bloom filter
        bf: Bloom = Bloom(max_set_size, error_rate)
        
        # adding all the keys from set k
        for key in sets_k[i]:
            # print(key)
            bf.add(key)
        
        for key in sets_k[i]:
            assert key in bf
        # print("finished adding keys to bf")
        # check number of keys from k' in bloom
        num_pos = 0
        start = time.time()
        for key in sets_k_prime[i]:
            if key in bf:
                num_pos += 1
        end = time.time()
        time_elapsed = end - start
                
        num_false_pos = num_pos - len(sets_k[i])
        print(str(error_rate) + ", " + str(max_set_size / 2) + ", " + str(num_false_pos) + ", " + str(time_elapsed) + ", " + str(bf.size_in_bits))




error rate, set size, num false positives, time elapsed, size in bits of bf
0.0078125, 500.0, 0, 0.0001552104949951172, 10104
0.0078125, 5000.0, 1, 0.0013911724090576172, 100992
0.0078125, 50000.0, 19, 0.014160871505737305, 1009888
0.0078125, 500000.0, 124, 0.14825916290283203, 10098872
0.0078125, 5000000.0, 3466, 2.243194580078125, 100988656
0.00390625, 500.0, 0, 0.00014781951904296875, 11544
0.00390625, 5000.0, 0, 0.001461029052734375, 115416
0.00390625, 50000.0, 1, 0.014629840850830078, 1154160
0.00390625, 500000.0, 45, 0.1555309295654297, 11541560
0.00390625, 5000000.0, 2834, 2.4593703746795654, 115415608
0.0009765625, 500.0, 0, 0.00015020370483398438, 14432
0.0009765625, 5000.0, 0, 0.0015497207641601562, 144272
0.0009765625, 50000.0, 0, 0.015507221221923828, 1442696
0.0009765625, 500000.0, 28, 0.16610240936279297, 14426952
0.0009765625, 5000000.0, 2581, 2.540933609008789, 144269504


In [5]:
# Task 2
print("set size, num false positives, time elapsed, length of mhpf")
for i in range(len(max_set_sizes)):
    max_set_size = max_set_sizes[i]
    # print("set size: " + str(set_size))
    # creating bloom filter
    table = BBHashTable()
    table.initialize(sets_k[i])
    for key in sets_k[i]:
        table[key] = 1    
    # adding all the keys from set k
    # for key in sets_k[i]:
    #     # print(key)
    #     bf.add(key)
    
    # for key in sets_k[i]:
    #     assert key in bf
    # print("finished adding keys to bf")
    # check number of keys from k' in bloom
    num_pos = 0
    start = time.time()
    for key in sets_k_prime[i]:
        if table[key] != None:
            num_pos += 1
    end = time.time()
    time_elapsed = end - start
            
    num_false_pos = num_pos - len(sets_k[i])
    print(str(max_set_size / 2) + ", " + str(num_false_pos) + ", " + str(time_elapsed) + ", " + str(len(table)))


set size, num false positives, time elapsed, length of mhpf
500.0, 0, 0.003194570541381836, 500
5000.0, 0, 0.025622129440307617, 5000
50000.0, 0, 0.2317214012145996, 50000
500000.0, 26, 2.3670687675476074, 500000
5000000.0, 2551, 24.392863988876343, 5000000


In [7]:
# Task 3
print("set size, num false positives, time elapsed, length of fingerprint, bf size in bits, fingerprint num bits")
for i in range(len(max_set_sizes)):
    max_set_size = max_set_sizes[i]
    # print("set size: " + str(set_size))
    # creating mphf
    mph = bbhash.PyMPHF(sets_k[i], max_set_size, 1, 1.0)
    bf: Bloom = Bloom(max_set_size, pow(2, -10))
    for key in sets_k[i]:
        bf.add(mph.lookup(key))    
    for num_bits in finger_print_bits:
        mod = 2 ** (num_bits)
        finger_print = {}
        for key in sets_k[i]:
            finger_print[mph.lookup(key)] = (key % mod)
        # print(finger_print)
        num_pos = 0
        start = time.time()
        for key in sets_k_prime[i]:
            if (mph.lookup(key) != None) and (mph.lookup(key) in bf) and (finger_print[mph.lookup(key)] == (key % mod)):
                num_pos += 1
        end = time.time()
        time_elapsed = end - start
                
        num_false_pos = num_pos - len(sets_k[i])
        print(str(max_set_size / 2) + ", " + str(num_false_pos) + ", " + str(time_elapsed) + ", "  + str(len(finger_print)) + ", " + str(bf.size_in_bits) + ", " + str(num_bits))


set size, num false positives, time elapsed, length of fingerprint, bf size in bits, fingerprint num bits
500.0, 4, 0.0004572868347167969, 500, 14432, 7
500.0, 2, 0.00045108795166015625, 500, 14432, 8
500.0, 1, 0.0004596710205078125, 500, 14432, 10
5000.0, 24, 0.005210161209106445, 5000, 144272, 7
5000.0, 11, 0.004811763763427734, 5000, 144272, 8
5000.0, 1, 0.005042076110839844, 5000, 144272, 10
50000.0, 218, 0.051125526428222656, 50000, 1442696, 7
50000.0, 98, 0.051438331604003906, 50000, 1442696, 8
50000.0, 25, 0.05370330810546875, 50000, 1442696, 10
500000.0, 2128, 0.6032590866088867, 499989, 14426952, 7
500000.0, 1091, 0.6040787696838379, 499989, 14426952, 8
500000.0, 280, 0.6141047477722168, 499989, 14426952, 10
5000000.0, 23434, 7.910686016082764, 4998815, 144269504, 7
5000000.0, 13026, 7.867715835571289, 4998815, 144269504, 8
5000000.0, 5192, 8.042538166046143, 4998815, 144269504, 10
